In [86]:
from hazm.Embedding import SentEmbedding, WordEmbedding
from hazm import PersicaReader, word_tokenize
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
persikaReader = PersicaReader('/repo/ebi/persica/persica.csv')

In [3]:
word2vec = WordEmbedding(model_path='/repo/ebi/embedding/training_files/fasttextV2_skipgram_dim300_epoch30_naabData0to35.bin',
                        model_type='fasttext')

tcmalloc: large alloc 5128601600 bytes == 0x1005c000 @ 
tcmalloc: large alloc 2728599552 bytes == 0x141b68000 @ 
tcmalloc: large alloc 2728599552 bytes == 0x1eddec000 @ 
tcmalloc: large alloc 2400002048 bytes == 0x29081e000 @ 
tcmalloc: large alloc 2728599552 bytes == 0x32de82000 @ 


In [4]:
sent2vec = SentEmbedding('/repo/ebi/embedding/training_files/trained_models/sent2vec/sent2vec-naab.model')

In [79]:
data = pd.DataFrame([[item['text'], item['category2']] for item in persikaReader.docs()],
                    columns = ['text', 'category'])

data = data[data['text'] != ''].reset_index(drop = True)
data

,text,category
0,وزير علوم در جمع استادان نمونه كشور گفت: از اس...,آموزشي
1,به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و ك...,آموزشي
3,محمدتقي علوي يزدي، مجري اين طرح پژوهشي در اين‌...,آموزشي
4,دوره آموزشي سيره نبوي بعنوان يكي از رشته‌هاي ج...,آموزشي
...,...,...
10983,به گزارش ايسنا، در كوران مسابقات دوچرخه سواري ...,ورزشي
10984,محمد دليريان در گفت‌وگو با خبرنگار كشتي خبرگزا...,ورزشي
10985,به گزارش خبرگزاري دانشجويان ايران (ايسنا)، در ...,ورزشي
10986,به گزارش خبرگزاري دانشجويان ايران (ايسنا)، اير...,ورزشي


In [80]:
data['category'].value_counts()

آموزشي        1000
اقتصادي       1000
بهداشتي       1000
مذهبي         1000
اجتماعي        999
فرهنگي         999
ورزشي          999
تاريخي         998
سياسي          998
فقه و حقوق     998
علمي           997
Name: category, dtype: int64

In [81]:
labels = data['category'].unique()
d = dict(zip(labels, range(len(labels))))
data['label'] = data['category'].map(d)
data

,text,category,label
0,وزير علوم در جمع استادان نمونه كشور گفت: از اس...,آموزشي,0
1,به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ...,آموزشي,0
2,نتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و ك...,آموزشي,0
3,محمدتقي علوي يزدي، مجري اين طرح پژوهشي در اين‌...,آموزشي,0
4,دوره آموزشي سيره نبوي بعنوان يكي از رشته‌هاي ج...,آموزشي,0
...,...,...,...
10983,به گزارش ايسنا، در كوران مسابقات دوچرخه سواري ...,ورزشي,10
10984,محمد دليريان در گفت‌وگو با خبرنگار كشتي خبرگزا...,ورزشي,10
10985,به گزارش خبرگزاري دانشجويان ايران (ايسنا)، در ...,ورزشي,10
10986,به گزارش خبرگزاري دانشجويان ايران (ايسنا)، اير...,ورزشي,10


In [82]:
from collections import defaultdict
from sklearn import metrics

evaluations = []
evaluations_std = []


def fit_and_evaluate(km, X, labels, name=None, n_runs=5):
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        km.fit(X)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
        
    evaluation = {
        "estimator": name,
    }
    evaluation_std = {
        "estimator": name,
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

In [83]:
kmeans = KMeans(
    n_clusters=len(labels),
    n_init=5
)

fit_and_evaluate(kmeans, [sent2vec[item] for item in data['text']], labels=list(data['label']), name="KMeans on sent2vec")


Homogeneity: 0.129 ± 0.014
Completeness: 0.167 ± 0.016
V-measure: 0.145 ± 0.015
Adjusted Rand-Index: 0.038 ± 0.005
Silhouette Coefficient: 0.006 ± 0.002


In [85]:
kmeans = KMeans(
    n_clusters=len(labels),
    n_init=5
)

mean_embed = []
for sent in tqdm(data['text']):
    embeds = [word2vec[word] for word in word_tokenize(sent)]   
    mean_embed.append(np.array(embeds).mean(axis=0))

fit_and_evaluate(kmeans, mean_embed, labels=list(data['label']), name="KMeans on mean of the word2vec")


100%|██████████████████████████████████████████████████████████| 10988/10988 [01:10<00:00, 156.45it/s]


Homogeneity: 0.103 ± 0.013
Completeness: 0.125 ± 0.013
V-measure: 0.113 ± 0.013
Adjusted Rand-Index: 0.062 ± 0.007
Silhouette Coefficient: 0.030 ± 0.002


In [91]:
kmeans = KMeans(
    n_clusters=len(labels),
    n_init=5
)

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
)

X_tfidf = vectorizer.fit_transform(data['text'])

fit_and_evaluate(kmeans, X_tfidf, labels=list(data['label']), name="KMeans on TfIdf algourithm")

Homogeneity: 0.408 ± 0.021
Completeness: 0.483 ± 0.023
V-measure: 0.442 ± 0.021
Adjusted Rand-Index: 0.217 ± 0.015
Silhouette Coefficient: 0.009 ± 0.001


<1x19060 sparse matrix of type '<class 'numpy.float64'>'
	with 177 stored elements in Compressed Sparse Row format>

In [41]:
mean_embed[0]

array([-1.15658075e-01,  3.30005400e-03,  1.19833916e-01,  7.61972442e-02,
       -8.86164233e-02,  2.89645325e-02,  2.83834618e-02, -1.17622614e-02,
       -1.34554923e-01,  4.75362912e-02, -3.69492956e-02, -5.96575662e-02,
        1.33599594e-01,  1.20970324e-01,  1.26907751e-01, -1.19320258e-01,
        1.16069943e-01,  1.08110838e-01,  2.38949254e-01, -4.55013737e-02,
       -4.12124116e-03, -9.43369046e-02, -1.23843521e-01,  8.81815925e-02,
        1.56715944e-01, -1.55732796e-01, -1.00387439e-01, -3.52374762e-02,
        9.44543555e-02, -3.94181199e-02,  7.12030381e-02,  3.75654958e-02,
        4.20922600e-02,  1.34677783e-01,  1.01551285e-03, -1.47273513e-02,
        4.21685018e-02,  1.18819937e-01,  3.41322720e-02, -1.03734680e-01,
       -4.71221469e-02, -1.22493459e-02, -8.30951929e-02,  2.25746259e-03,
        4.04331312e-02, -8.62189233e-02, -1.84172988e-02, -9.22953486e-02,
       -6.51553497e-02, -1.51986172e-02,  1.59867015e-02,  8.35921019e-02,
        1.28735945e-01,  

In [92]:
kmeans = KMeans(n_clusters=11, init='k-means++', random_state=10)
answerData = kmeans.fit_predict(data)

In [93]:
answerDataFrame = pd.DataFrame(answerData)

In [94]:
answerDataFrame.value_counts()

3     3914
1     2872
9      871
7      816
5      808
10     603
8      367
0      331
4      194
6      126
2       97
dtype: int64

In [70]:
answerData

array([0, 2, 2, ..., 2, 2, 6], dtype=int32)

In [71]:
classes = []
for i in range(kmeans.n_clusters):
    classes.append(list(np.where(answerData == i)[0])) 
    

In [72]:
sents_array = np.array(sents)

tcmalloc: large alloc 1438015488 bytes == 0xd7b74000 @ 


In [75]:
sents_array[classes[9]][1:4]

array(['پرونده دانشگاه غيرمجاز هاوايي شايد يكي از مهمترين پرونده\u200cهاي حقوقي در عرصه آموزش عالي كشور به شمار مي\u200cرود، پرونده\u200cاي كه از سال 1380 با شكايت وزارت علوم مطرح شد. اين پرونده در دوره وزارت دكتر مصطفي معين تشكيل شد و در دوره دكتر جعفر توفيقي وزير سابق پيگيري شد و پيگيري آن تا به امروز در دوره وزارت دكتر زاهدي نيز ادامه يافت. به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ايران (ايسنا)، دانشگاه غيرمجاز هاوايي كه در سال \u200c1372 تاسيس شد از سال \u200c1374 فعاليتهاي خود را در قالب پذيرش دانشجو و آموزش در مقاطع مختلف آموزش عالي از راه دور آغاز كرد، اقدامي كه در سال 1379 از سوي وزارت علوم غيرقانوني و مدارك آن غيرمعتبر اعلام شد. دانشگاه هاوايي در آن زمان كه مورد شكايت وزارت علوم قرار گرفت، داراي بيش از 5 هزار دانشجو و 2 هزار و 500 فارغ\u200cالتحصيل بود كه همگي آنها شناسايي شده\u200cاند، هر چند برخي از آنها &nbsp;از قضات، معاونان وزرا و مسوولان &nbsp;بودند، مساله\u200cاي كه &nbsp;نگراني &nbsp;وزارت علوم نسبت به احتمال &nbsp;تاثيرگذاري و اخلال در روند بررسي پرونده را در برد

In [76]:
next(persikaReader.docs())

{'id': 843656,
 'title': 'وزير علوم درجمع استادان نمونه: سن بازنشستگي استادان نمونه به 70 سال افزايش مي\u200cيابد دانشگاه بايد مهد چالشهاي گفتماني و خط دهنده و برنامه\u200cريز جريانات سياسي باشد',
 'text': 'وزير علوم در جمع استادان نمونه كشور گفت: از استادان نمونه كشور انتظار مي\u200cرود كه رويكرد دانايي محوري و گفتمان علمي را به عنوان يك بحث فرهنگي در دانشگاهها توسعه و رونق بخشند. به گزارش سرويس صنفي آموزشي خبرگزاري دانشجويان ايران (ايسنا)، دكتر محمد مهدي زاهدي در اولين مجمع عمومي استادان نمونه دانشگاه\u200cهاي سراسر كشور كه در دانشگاه تهران برگزار شد، افزود: توصيه ما در جهت تلاش براي دانايي محوري و توسعه گفتمان علمي به معني عدم تمايل به مباحث سياسي نيست؛ بلكه برعكس، دانشگاه بايد مهد چالشهاي گفتماني باشد ولي اين امر، بدان معني نيست كه دانشگاه، ابزار دست سياسيون قرار بگيرد. وي تأكيد كرد: دانشگاه نه تنها نبايد تحت تأثير القائات سياسي قرار بگيرد؛ بلكه بايد خط دهنده و برنامه\u200cريز جريانات سياسي باشد و مهمترين عنصر پياده شدن اين آرمان، دانشجويان و اعضاي هيات علمي دانشگاهها و در رأس آنها

In [83]:
pd.DataFrame(tags1).value_counts()

تاريخ-                      999
آموزشي-                     997
حج و زيارت-                 986
فقه و حقوق-حقوق اجتماعي-    553
بهداشت و درمان-عمومي-       442
                           ... 
اديان-                        1
فناوري استراتژيك-نظامي-       1
افغانستان-                    1
همسايگان ايران-               1
فرهنگ و هنر-مکتب اصفهان-      1
Length: 97, dtype: int64

In [82]:
pd.DataFrame(tags2).value_counts()

آموزشي        1000
اجتماعي       1000
اقتصادي       1000
بهداشتي       1000
سياسي         1000
علمي          1000
فرهنگي        1000
فقه و حقوق    1000
مذهبي         1000
ورزشي         1000
تاريخي         999
dtype: int64